# 机器学习练习 4 - 神经网络

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed()
data = loadmat('ex4data1.mat')
X = data['X']
y = data['y']
X = pd.DataFrame(X)
y = pd.DataFrame(y)
X = X.sample(n=500, replace=False, random_state=1, axis=0)
y = y.sample(n=500, replace=False, random_state=1, axis=0)
hidden_num = 25
alpha = 1

In [3]:
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y = y_onehot  
y_onehot.shape

(500, 10)

In [4]:
theta = (np.random.random(size = hidden_num * (X.shape[1] + 1) + y.shape[1] * (hidden_num + 1)) - 0.5) * 0.25

# Backpropagation

In [16]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def sigmoidDerivative(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))
    

In [30]:
def BP(theta, X, y, hidden_num, alpha):

    '''
    Matrixing
    '''
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    '''
    Expanding theta
    '''
    theta1 = np.reshape(theta[:, :hidden_num * (X.shape[1] + 1)], (hidden_num, X.shape[1] + 1))
    theta2 = np.reshape(theta[:, hidden_num * (X.shape[1] + 1):], (y.shape[1] , (hidden_num + 1)))

    '''
    Forward Propagation
    '''
    z1 = X
    a1 = np.insert(z1, 0, values=np.ones(X.shape[0]), axis=1)
    z2 = a1 * theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, values=np.ones(X.shape[0]), axis=1)
    z3 = a2 * theta2.T
    a3 = sigmoid(z3)

    '''
    Cost Function
    '''
    cos =np.multiply(-y, np.log(sigmoid(a2 * theta2.T))) - np.multiply((1 - y), np.log(1 - sigmoid(a2 * theta2.T)))
    reg = np.sum(np.power(theta2, 2)) * (alpha / (2 * len(a2))) + np.sum(np.power(theta1, 2)) * (alpha / len(z1))
    Cost = np.sum(cos) / len(X) + reg
    
    
    '''
    BackPropagation
    '''
    delta1 = np.zeros((hidden_num, X.shape[1] + 1))
    delta2 = np.zeros((y.shape[1], hidden_num + 1))
    for i in range(X.shape[0]):
        x_i = X[i, :]
        y_i = y[i, :]

        a1 = a1[i, :]
        a2 = a2[i, :]
        a3 = a3[i, :]
        z2 = np.insert(z2[i, :], 0, values=np.ones(1), axis=1) #(1, 26)

        d3 = y_i - a3 #(1, 10)
        d2 = np.multiply((theta2.T * d3.T).T, sigmoidDerivative(z2)) #(1, 26)
        d2 = d2[:, 1:]
        #z2本不要加偏置

        delta1 += (d2.T * a1)
        delta2 += (d3.T * a2)

        # if i % 50 == 0:
            # print(delta1, delta2)

    delta1 = delta1 / len(X)
    delat2 = delat2 / len(X)
    # print(delta1, delat2)

    grad = np.concatenate(np.ravel(delta1), np.ravel(delat2))
    return Cost, grad



反向传播计算的最难的部分（除了理解为什么我们正在做所有这些计算）是获得正确矩阵维度。 顺便说一下，你容易混淆了A * B与np.multiply（A，B）使用。 基本上前者是矩阵乘法，后者是元素乘法（除非A或B是标量值，在这种情况下没关系）
     

In [31]:
J, grad = BP(theta, X, y, hidden_num, alpha)
grad.shape

IndexError: index 1 is out of bounds for axis 0 with size 1